In [2]:
import numpy as np
from numba import jit, prange
import timeit

In [3]:
def kron_gates_l(single_gates):
    result = single_gates[0]

    for gate in single_gates[1:]:
        result = np.kron(result, gate)

    return result


@jit(parallel=True)
def kron_neighbours_even(single_gates):

    l, dims, _ = single_gates.shape

    double_gates = np.zeros((l // 2, dims**2, dims**2), dtype=np.complex128)

    for i in prange(0, l // 2):
        double_gates[i, :, :] = np.kron(single_gates[i * 2], single_gates[i * 2 + 1])

    return double_gates


def kron_gates_t(single_gates):
    """Recursively multiply the neighbouring gates.
    When the block size gets below the turnover point the linear
    kron_gates_l is used as it is more efficient in this usecase."""
    TURNOVER = 3

    l = len(single_gates)

    if l > TURNOVER:
        if l % 2 == 0:
            return kron_gates_t(kron_neighbours_even(single_gates))
        return np.kron(
            kron_gates_t(kron_neighbours_even(single_gates[:-1, :, :])),
            single_gates[-1],
        )

    return kron_gates_l(single_gates)

In [11]:
kron_gates_t(np.random.rand(5, 2, 2))

array([[4.41580700e-02+0.j, 1.04774069e-01+0.j, 4.88480171e-02+0.j, ...,
        1.17959927e-01+0.j, 5.49955594e-02+0.j, 1.30488234e-01+0.j],
       [2.93199190e-02+0.j, 1.86232076e-02+0.j, 3.24339335e-02+0.j, ...,
        2.09669457e-02+0.j, 3.65157569e-02+0.j, 2.31938063e-02+0.j],
       [5.42458706e-02+0.j, 1.28709443e-01+0.j, 9.25730850e-02+0.j, ...,
        1.44907578e-01+0.j, 1.04223444e-01+0.j, 2.47291478e-01+0.j],
       ...,
       [4.44221291e-04+0.j, 2.82157169e-04+0.j, 4.91401214e-04+0.j, ...,
        3.89768011e-02+0.j, 6.78814841e-02+0.j, 4.31164552e-02+0.j],
       [8.21870301e-04+0.j, 1.95005569e-03+0.j, 1.40255965e-03+0.j, ...,
        2.69377997e-01+0.j, 1.93747650e-01+0.j, 4.59706000e-01+0.j],
       [5.45702532e-04+0.j, 3.46615268e-04+0.j, 9.31266591e-04+0.j, ...,
        4.78809537e-02+0.j, 1.28643879e-01+0.j, 8.17110603e-02+0.j]])

In [5]:
def rz(thetas):

    zero = np.zeros(thetas.shape)
    exp_m_theta = np.exp(-1j * thetas / 2)
    exp_theta = np.exp(1j * thetas / 2)

    single_gates = np.einsum(
        "ijk->kji", np.array([[exp_m_theta, zero], [zero, exp_theta]]), optimize="greedy",
    order="C"
    )

    u_gates = kron_gates_t(single_gates)

    return u_gates

In [6]:
from pykronecker import KroneckerProduct
def rz3(thetas):

    zero = np.zeros(thetas.shape)
    exp_m_theta = np.exp(-1j * thetas / 2)
    exp_theta = np.exp(1j * thetas / 2)

    single_gates = np.einsum(
        "ijk->kji",
        np.array([[exp_m_theta, zero], [zero, exp_theta]]),
        optimize="greedy",
        # order="C",
    )

    u_gates = KroneckerProduct(single_gates)

    return u_gates

Using NumPy backend


In [7]:
def rz2(thetas):

    gate = np.array([1])

    for theta in thetas:
        single_gate = np.array([[np.exp(-1j * theta / 2), 0], [0, np.exp(1j * theta / 2)]])
        gate = np.kron(gate, single_gate)

    return gate

In [8]:
%timeit  rz3(np.ones(7)) @ rz3(np.ones(7)+1) @ rz3(np.ones(7)+2 ).to_array()

4.18 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit rz(np.ones(7)) @ rz(np.ones(7)+1) @ rz(np.ones(7)+2 )

1.38 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit rz2(np.ones(7))

347 µs ± 7.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
